In [ ]:
import sys
sys.path.append("lib/python3.7/site-packages/")
# from pyclustertend import hopkins
from sklearn.preprocessing import scale
import numpy as np
import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import TfidfVectorizer 

In [ ]:
N_DOCUMENTS = 517402
# START = 100000

print("Loading dataset")

#load clean emails
X = list()
for i in range(1, N_DOCUMENTS):
    f = open("clean/"+str(i)+".txt", "r")
    X.append(f.read())

print("TFIDF")
tfidf_vectorizer=TfidfVectorizer(input = 'content', use_idf=True)
tfidf_vectors=tfidf_vectorizer.fit_transform(X)
tfidf_vectors = tfidf_vectors.toarray()

In [ ]:
# Finding Cluster Tendency: the following code has been taken from 
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
import numpy as np
from math import isnan

def hopkins(X):
    d = X.shape[1]
    #d = len(vars) # columns
    n = len(X) # rows
    m = int(0.01 * n) # heuristic from article [1]
    print("NearestNeighbors")
    nbrs = NearestNeighbors(n_neighbors=1).fit(X)
 
    rand_X = sample(range(0, n, 1), m)
 
    ujd = []
    wjd = []
    print("Starting loop, m = " + str(m))
    for j in range(0, m):
        if not (j%250):
            print(j)
        u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X[rand_X[j]].reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])
 
    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if isnan(H):
        print(ujd, wjd)
        H = 0
    with open("hopkins_score.txt", "w") as outf:
        outf.write(str(H))
    print(H)

print("Calling function")
hopkins(tfidf_vectors)